### Imports & Definitions

In [1]:
import sys
sys.path.append("../../utils")
from definitions import *

In [5]:
import os
from huggingface_hub import login
login(os.environ['hf-read-token'])

In [18]:
MODEL_EXPERIMENT_NAME = "lora_v1"
MODEL_MAX_LENGTH = 16384
INPUT_COLUMNS = ["test_row", "id", "abstract"] # ['id', 'abstract', 'diploma', 'begin', 'raw_model_v2', 'learnt']

### Load inputs & assessors dataset

In [20]:
inputs = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("metrics/diplomas_asessors_questions/inputs.csv"), usecols=INPUT_COLUMNS)
inputs = inputs.set_index("test_row")
inputs

id                                           abstract
test_row                                                          
12        45042  В этой работе мы строим правую трансферную мод...
25        45043  Пусть 𝐾 выпуклое тело в ℝ^𝑛. Определим 𝑑𝑛,𝑛−1(...
37        45044  Работа посвящена повышению производительности ...
101       45046  В работе мы обобщаем результаты об энергии нат...
152       45047  В рамках данной работы рассматривается подход ...
...         ...                                                ...
1062      45131  В настоящей работе были рассмотрены две задачи...
1064      45132  В данной работе исследовались MS/MS спектры, с...
1079      45133  В выпускной работе описываются триангуляции ве...
1209      45135  В рамках данной работы было разработано програ...
1238      45137  В данной работе представлены реализация фронте...

[70 rows x 2 columns]

In [6]:
with open(ARTIFACTS_DIR_PATH.joinpath("datasets/diplomas_asessors_questions/mcs_df_human_filled_processed.json"), "r") as f:
    asessors_dataset = json.load(f)
asessors_dataset

[{'instruction': 'Задание содержит контексти и вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\nКонтекст: {context}\nВопрос: {text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:',
  'inputs': {'text': 'Какой объект строят в этой работе?',
   'option_a': 'Левую трансферную модельную структуру',
   'option_b': 'Моноид в категории эндофункторов',
   'option_c': 'В этой работе ничего не строят',
   'option_d': 'Правую трансферную модельную структуру',
   'subject': 'Математика и компьютерные науки',
   'context': 'В этой работе мы строим правую трансферную модельную структуру на категории алгебр обогащенной алгебраической теории в обогащенной модельной категории, удовлетворяющей некоторым свойствам. Также мы даем несколько общих конструкций генерирующих замкнутые симметричные моноидальные локально конечно представимые категори, которые служат базой обогащения.'},
  'outputs': 'D',
  'meta': {'a

### Load model

##### Baseline

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[LLAMA_2_7B], 
    cache_dir=CACHE_DIR, 
    torch_dtype=torch.bfloat16,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print("Loaded model")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[LLAMA_2_7B],
    cache_dir=CACHE_DIR,
    model_max_length=MODEL_MAX_LENGTH,
    padding_side="right",
    use_fast=True,
)

print("Loaded tokenizer")

##### Experiment

In [ ]:
import sys
import io
import os
import copy
import json
import math
import logging
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer, DataCollatorForLanguageModeling
from llama_attn_replace_sft import replace_llama_attn
from gptneox_attn_replace import replace_gpt_neox_attn
from peft import LoraConfig, get_peft_model
from torch.distributed import barrier


model_name = LLAMA_2_7B

replace_llama_attn(True, False, inference=True)

# Set RoPE scaling factor
config = transformers.AutoConfig.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name],
    cache_dir=CACHE_DIR,
)

orig_rope_scaling = getattr(config, "rope_scaling", None)
if orig_rope_scaling is None:
    orig_rope_scaling = {"factor": 1}
orig_rope_scaling_factor = orig_rope_scaling["factor"] if "factor" in orig_rope_scaling.keys() else 1
orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len:
    orig_ctx_len *= orig_rope_scaling_factor
    if MODEL_MAX_LENGTH > orig_ctx_len:
        scaling_factor = float(math.ceil(MODEL_MAX_LENGTH / orig_ctx_len))
        config.rope_scaling = {"type": "linear", "factor": scaling_factor}

print("Created config")

# Load model and tokenizer
model = transformers.AutoModelForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name], 
    cache_dir=CACHE_DIR, 
    config=config,
    torch_dtype=torch.bfloat16,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print("Loaded model")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name],
    cache_dir=CACHE_DIR,
    model_max_length=MODEL_MAX_LENGTH,
    padding_side="right",
    use_fast=True,
)

print("Loaded tokenizer")

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"


special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN


def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg
        
smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)

In [ ]:
from peft import PeftModel

model_id = "nvdenisov2002/llama-longLoRA-v1"
model = PeftModel.from_pretrained(model, model_id)

### Evaluate model on inputs

In [21]:
inputs.columns

Index(['id', 'abstract'], dtype='object')

In [ ]:
from mera_helpers import construct_prompt
from llm_helpers import calculate_token_interest_probs, get_answer

for cur_input in inputs.columns:
    if cur_input == "id":
        continue
    metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{MODEL_EXPERIMENT_NAME}_{cur_input}_appended/{LLAMA_2_7B}/")
    metric_dir_path.mkdir(exist_ok=True, parents=True)
    for row in tqdm(asessors_dataset, desc="Rows..."):
        x = copy.deepcopy(row)
        x['inputs']['context'] = inputs[cur_input].loc[int(x['meta']['id'])]
        q = construct_prompt(x)
        probs = calculate_token_interest_probs(q, tokenizer, model)
        probs_entry = {
            "probs": probs,
            "meta": row["meta"],
        }
        a = get_answer(probs)
        a_entry = {
            "answer": a,
            "meta": row["meta"],
        }
        with open(metric_dir_path.joinpath("probs_appended.jsons"), "a") as f:
            json.dump(probs_entry, f, ensure_ascii=False)
            f.write("\n")
        with open(metric_dir_path.joinpath("answers_appended.jsons"), "a") as f:
            json.dump(a_entry, f, ensure_ascii=False)
            f.write("\n")

In [ ]:
rows = []
for cur_input in inputs.columns:
    metric_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{MODEL_EXPERIMENT_NAME}_{cur_input}_appended/{LLAMA_2_7B}/")
    some_path = metric_path.joinpath("answers_appended.jsons")
    with open(some_path, "r") as f:
        answers = [json.loads(x) for x in f.readlines()]
    pred = [x["answer"] for x in answers]
    true = [x["outputs"] for x in asessors_dataset]
    rows.append({
        "model": MODEL_EXPERIMENT_NAME,
        "subset": "asessors_questions",
        "split": cur_input,
        "accuracy_score": accuracy_score(true, pred),
    })
asessors_df = pd.DataFrame(rows)
asessors_df

### Write results

In [ ]:
asessors_df.to_csv(METRICS_DIR_PATH.joinpath(f"assessors_{MODEL_EXPERIMENT_NAME}.csv"))